In [2]:
import pandas as pd
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives import padding
from cryptography.hazmat.backends import default_backend
import base64
import os

def encrypt_aes(plain_text, key):
    # Pad plain_text to be a multiple of 16 bytes
    padder = padding.PKCS7(algorithms.AES.block_size).padder()
    padded_plain_text = padder.update(plain_text.encode()) + padder.finalize()

    # Generate a random IV
    iv = os.urandom(16)

    # Create AES cipher object
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
    encryptor = cipher.encryptor()

    # Encrypt the data
    encrypted_text = encryptor.update(padded_plain_text) + encryptor.finalize()

    # Encode the encrypted data and IV in base64 for easier storage
    encrypted_text_b64 = base64.b64encode(encrypted_text).decode('utf-8')
    iv_b64 = base64.b64encode(iv).decode('utf-8')

    return encrypted_text_b64, iv_b64

def process_csv(input_csv, output_csv, key):
    # Read the CSV file
    df = pd.read_csv(input_csv)

    # Encrypt all columns
    encrypted_df = pd.DataFrame()
    for column in df.columns:
        encrypted_column = []
        for value in df[column].astype(str):  # Convert all values to string
            encrypted_value, _ = encrypt_aes(value, key)
            encrypted_column.append(encrypted_value)
        encrypted_df[column] = encrypted_column

    # Write the encrypted DataFrame to a new CSV file
    encrypted_df.to_csv(output_csv, index=False)
    print(f"Encrypted CSV saved to {output_csv}")

# Example usage
key = os.urandom(32)  # 32 bytes key for AES-256
input_csv = 'dataset_updated.csv'
output_csv = 'encrypted_output.csv'

process_csv(input_csv, output_csv, key)


Encrypted CSV saved to encrypted_output.csv


In [5]:

import pandas as pd
import hashlib

def hash_md5(plain_text):
    # Create an MD5 hash object
    md5 = hashlib.md5()
    # Update the hash object with the bytes of the plaintext
    md5.update(plain_text.encode('utf-8'))
    # Return the hexadecimal representation of the hash
    return md5.hexdigest()

def process_csv(input_csv, output_csv):
    # Read the CSV file
    df = pd.read_csv(input_csv)

    # Hash all columns
    hashed_df = pd.DataFrame()
    for column in df.columns:
        hashed_column = []
        for value in df[column].astype(str):  # Convert all values to string
            hashed_value = hash_md5(value)
            hashed_column.append(hashed_value)
        hashed_df[column] = hashed_column

    # Write the hashed DataFrame to a new CSV file
    hashed_df.to_csv(output_csv, index=False)
    print(f"Hashed CSV saved to {output_csv}")

# Example usage
input_csv = 'encrypted_output.csv'
output_csv = 'your_data.csv'
process_csv(input_csv, output_csv)



Hashed CSV saved to your_data.csv
